<a href="https://colab.research.google.com/github/hentzrafael/tcc2-carbon-estimator/blob/main/carbon_unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rasterio mlflow tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 199.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 137.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 129.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 180.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.

In [ ]:
input_file =  '/content/drive/MyDrive/TCC/data/clipped2022.tif'
target_file = '/content/drive/MyDrive/TCC/data/ceda2022Amazon_norm.tif'
patch_size = 256
stride = 64
batch_size = 32
epochs = 50

In [ ]:
import torch
torch.cuda.empty_cache()
import mlflow
import mlflow.pytorch
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import torchvision.utils as vutils
import os
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'mlflow'

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
import rasterio
from rasterio.windows import Window


class GeoTiffPatchDatasetShuffled(Dataset):
    _cached_inputs = None
    _cached_targets = None
    _cached_meta = None

    def __init__(self, input_path, target_path, patch_size=256, stride=256,
                 split="train", seed=42, transform=None):
        """
        input_path: path to multi-band input GeoTIFF (5 bands)
        target_path: path to single-band target GeoTIFF (carbon map)
        patch_size: size of square patches
        stride: stride to slide patches (can be < patch_size for overlap)
        split: "train", "val" or "test"
        seed: random seed for reproducible shuffling
        transform: optional transform(input, target, mask)
        """
        self.input_path = input_path
        self.target_path = target_path
        self.patch_size = patch_size
        self.stride = stride
        self.split = split
        self.seed = seed
        self.transform = transform

        if GeoTiffPatchDatasetShuffled._cached_inputs is None:
          print("Loading GeoTIFFs into memory (first time only)...")
          with rasterio.open(input_path) as src_in:
              self.input_img = src_in.read().astype(np.float32)
              self.width, self.height = src_in.width, src_in.height
              self.nodata_in = src_in.nodata

          with rasterio.open(target_path) as src_tgt:
              self.target_img = src_tgt.read(1).astype(np.float32)
              self.nodata_tgt = src_tgt.nodata

          GeoTiffPatchDatasetShuffled._cached_inputs = self.input_img
          GeoTiffPatchDatasetShuffled._cached_targets = self.target_img
          GeoTiffPatchDatasetShuffled._cached_meta = (self.width, self.height, self.nodata_in, self.nodata_tgt)
        else:
          print("Using cached GeoTIFFs in memory.")
          self.input_img = GeoTiffPatchDatasetShuffled._cached_inputs
          self.target_img = GeoTiffPatchDatasetShuffled._cached_targets
          self.width, self.height, self.nodata_in, self.nodata_tgt = GeoTiffPatchDatasetShuffled._cached_meta

        # Precompute patches
        patches = []
        for top in range(0, self.height - patch_size + 1, stride):
            for left in range(0, self.width - patch_size + 1, stride):
                input_patch = self.input_img[:, top:top+patch_size, left:left+patch_size]
                target_patch = self.target_img[top:top+patch_size, left:left+patch_size]

                # Normalize inputs
                input_patch = input_patch.copy()
                input_patch[0:3] /= 255.0  # RGB
                input_patch[3:] = (input_patch[3:] + 1.0) / 2.0  # NDVI, EVI
                input_patch = np.clip(input_patch, 0.0, 1.0)

                # Mask
                mask = np.ones(target_patch.shape, dtype=bool)
                if self.nodata_in is not None:
                    for b in range(input_patch.shape[0]):
                        mask &= (input_patch[b] != self.nodata_in)
                if self.nodata_tgt is not None:
                    mask &= (target_patch != self.nodata_tgt)

                target_patch = np.where(mask, target_patch, 0.0)

                patches.append((input_patch, target_patch, mask.astype(np.float32)))

        # Shuffle patches with fixed seed
        rng = np.random.default_rng(seed)
        rng.shuffle(patches)

        # Split into train/val/test (70/15/15)
        n_total = len(patches)
        n_train = int(0.7 * n_total)
        n_val = int(0.15 * n_total)

        if split == "train":
            self.patches = patches[:n_train]
        elif split == "val":
            self.patches = patches[n_train:n_train + n_val]
        elif split == "test":
            self.patches = patches[n_train + n_val:]
        else:
            self.patches = patches

    def __len__(self):
        return len(self.patches)

    def __getitem__(self, idx):
        input_patch, target_patch, mask = self.patches[idx]

        input_tensor = torch.from_numpy(input_patch)
        target_tensor = torch.from_numpy(target_patch).unsqueeze(0)
        mask_tensor = torch.from_numpy(mask).unsqueeze(0)

        if self.transform:
            input_tensor, target_tensor, mask_tensor = self.transform(
                input_tensor, target_tensor, mask_tensor
            )

        return input_tensor, target_tensor, mask_tensor


In [ ]:
import os
import argparse
import numpy as np
import rasterio
from rasterio.windows import Window
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


# -----------------------
# Patch Dataset for two large GeoTIFFs
# -----------------------
class GeoTIFFPatchDataset(Dataset):
    def __init__(self, input_path, target_path, patch_size=256, stride=256, transform=None):
        """
        input_path: path to multi-band input GeoTIFF (5 bands)
        target_path: path to single-band target GeoTIFF (carbon map)
        patch_size: size of square patches
        stride: stride to slide patches (can be < patch_size for overlap)
        """
        self.input_path = input_path
        self.target_path = target_path
        self.patch_size = patch_size
        self.stride = stride
        self.transform = transform

        # Open datasets once for metadata
        with rasterio.open(input_path) as src_in, rasterio.open(target_path) as src_tgt:
            self.width = src_in.width
            self.height = src_in.height
            self.input_meta = src_in.meta
            self.target_meta = src_tgt.meta
            self.nodata_in = src_in.nodata
            self.nodata_tgt = src_tgt.nodata

        # Calculate patch coordinates
        self.patches = []
        for top in range(0, self.height - patch_size + 1, stride):
            for left in range(0, self.width - patch_size + 1, stride):
                self.patches.append((left, top))

    def __len__(self):
        return len(self.patches)

    def __getitem__(self, idx):
        left, top = self.patches[idx]

        # Read patch windows from input and target files
        with rasterio.open(self.input_path) as src_in:
            input_patch = src_in.read(window=Window(left, top, self.patch_size, self.patch_size)).astype(np.float32)  # shape (5, ph, pw)

        with rasterio.open(self.target_path) as src_tgt:
            target_patch = src_tgt.read(1, window=Window(left, top, self.patch_size, self.patch_size)).astype(np.float32)  # shape (ph, pw)

        # Normalize inputs
        # RGB: scale 0-255 to 0-1
        input_patch[0:3] /= 255.0
        # NDVI, EVI: from [-1,1] to [0,1]
        input_patch[3:] = (input_patch[3:] + 1.0) / 2.0
        input_patch = np.clip(input_patch, 0.0, 1.0)

        # Create mask for nodata pixels
        mask = np.ones(target_patch.shape, dtype=bool)
        if self.nodata_in is not None:
            for b in range(input_patch.shape[0]):
                mask &= (input_patch[b] != self.nodata_in)
        if self.nodata_tgt is not None:
            mask &= (target_patch != self.nodata_tgt)

        # Mask out nodata in target
        target_patch = np.where(mask, target_patch, 0.0)

        # To torch tensors
        input_tensor = torch.from_numpy(input_patch)
        target_tensor = torch.from_numpy(target_patch).unsqueeze(0)
        mask_tensor = torch.from_numpy(mask.astype(np.float32)).unsqueeze(0)

        if self.transform:
            input_tensor, target_tensor, mask_tensor = self.transform(input_tensor, target_tensor, mask_tensor)

        return input_tensor, target_tensor, mask_tensor


# -----------------------
# U-Net model - reuse from before
# -----------------------
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)


class UNet(nn.Module):
    def __init__(self, in_channels=5, out_channels=1, features=[64, 128, 256, 512]):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(2, 2)

        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        for feature in reversed(features):
            self.ups.append(nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2))
            self.ups.append(DoubleConv(feature * 2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)
        self.final_conv = nn.Conv2d(features[0], out_channels, 1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)

        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx // 2]

            if x.shape != skip_connection.shape:
                x = nn.functional.interpolate(x, size=skip_connection.shape[2:])

            x = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx + 1](x)

        return self.final_conv(x)


# -----------------------
# Loss and training functions
# -----------------------
def masked_mse_loss(pred, target, mask):
    diff = (pred - target) * mask
    return torch.sum(diff ** 2) / torch.sum(mask)


def train_one_epoch(model, loader, optimizer, device):
    model.train()
    total_loss = 0.0
    for inputs, targets, masks in tqdm(loader):
        inputs, targets, masks = inputs.to(device), targets.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = masked_mse_loss(outputs, targets, masks)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)


def validate_one_epoch(model, loader, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for inputs, targets, masks in tqdm(loader):
            inputs, targets, masks = inputs.to(device), targets.to(device), masks.to(device)
            outputs = model(inputs)
            loss = masked_mse_loss(outputs, targets, masks)
            total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)


# -----------------------
# Main
# -----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")



Using device: cuda


In [ ]:
# Dataset and loader
train_dataset = GeoTiffPatchDatasetShuffled(input_file, target_file, split='train')
val_dataset = GeoTiffPatchDatasetShuffled(input_file, target_file, split='val')
test_dataset = GeoTiffPatchDatasetShuffled(input_file, target_file, split='test')

Loading GeoTIFFs into memory (first time only)...
Using cached GeoTIFFs in memory.
Using cached GeoTIFFs in memory.


In [ ]:

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)


# --- Setup experiment tracking ---
experiment_name = "Carbon_UNet_Tracking"
mlflow.set_experiment(experiment_name)

run_name = f"unet_run_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
log_dir = os.path.join("/content/drive/MyDrive/TCC/runs", run_name)
writer = SummaryWriter(log_dir=log_dir)

import io
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def figure_to_array(fig):
    """Convert a matplotlib figure to a NumPy RGB array."""
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    buf.seek(0)
    img = Image.open(buf).convert("RGB")
    return np.array(img)

def visualize_sample(inputs, targets, preds, step, max_samples=3):
    """Save sample patches (input RGB, target, prediction)"""
    os.makedirs("samples", exist_ok=True)
    samples_logged = 0

    for i in range(min(max_samples, inputs.size(0))):
        inp = inputs[i, :3].cpu().numpy().transpose(1, 2, 0)  # RGB bands
        inp = (inp - inp.min()) / (inp.max() - inp.min() + 1e-8)
        tgt = targets[i, 0].cpu().numpy()
        pred = preds[i, 0].cpu().numpy()

        fig, axs = plt.subplots(1, 3, figsize=(9, 3))
        axs[0].imshow(inp)
        axs[0].set_title("Input RGB")
        axs[1].imshow(tgt, cmap="viridis")
        axs[1].set_title("Target")
        axs[2].imshow(pred, cmap="viridis")
        axs[2].set_title("Prediction")
        for ax in axs: ax.axis("off")

        fig.tight_layout()
        path = f"samples/sample_{step}_idx{i}.png"
        plt.savefig(path)
        plt.close(fig)

        img_array = figure_to_array(fig)
        # Log image to TensorBoard
        writer.add_image(f"Samples/Epoch_{step}_Sample_{i}", img_array.transpose(2, 0, 1), global_step=step)

        # Also store the sample in MLflow
        mlflow.log_image(Image.fromarray(img_array), f"samples/Epoch_{step}.png")
        samples_logged += 1

    print(f"Logged {samples_logged} sample images for epoch {step}")



NameError: name 'DataLoader' is not defined

In [ ]:
pip install segmentation_models_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.6 MB/s eta 0:00:00


In [ ]:
with mlflow.start_run(run_name=run_name):
    mlflow.log_param("patch_size", patch_size)
    mlflow.log_param("stride", stride)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("learning_rate", 1e-4)
    # mlflow.log_param("model", "UNet_5_to_1")


    best_val_loss = float("inf")

    # model = UNet(in_channels=5, out_channels=1).to(device)

    import segmentation_models_pytorch as smp

    model = smp.DeepLabV3Plus(
      encoder_name="resnet101",        # backbone
      encoder_weights="imagenet",            # set to "imagenet" if you want pretrained weights
      in_channels=5,                   # your dataset has 5 input channels
      classes=1,                       # single regression/segmentation output
    )
    model = model.to(device)
    mlflow.log_param("model", "DeepLabV3Plus_ResNet101")

    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}/{epochs}")
        train_loss = train_one_epoch(model, train_loader, optimizer, device)
        print(f"Train Loss: {train_loss:.6f}")

        val_loss = validate_one_epoch(model, val_loader, device)
        print(f"Validation Loss: {val_loss:.6f}")

        writer.add_scalar("Loss/train", train_loss, epoch)
        writer.add_scalar("Loss/val", val_loss, epoch)
        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)

        if epoch % 5 == 0 or epoch == epochs:  # every 5 epochs or last
            model.eval()
            with torch.no_grad():
                val_inputs, val_targets, val_masks = next(iter(val_loader))
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                preds = model(val_inputs)
                visualize_sample(val_inputs, val_targets, preds, epoch)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_path = "/content/drive/MyDrive/TCC/deepLabV3.pt"
            torch.save(model.state_dict(), best_model_path)
            example = torch.randn(1, 5, patch_size, patch_size).cpu().numpy()
            mlflow.pytorch.log_model(model, name="model",input_example=example)
            mlflow.log_artifact(best_model_path)
            print(f"✅ Saved new best model: val_loss={best_val_loss:.6f}")

    mlflow.log_metric("best_val_loss", best_val_loss)

    print("Training complete.")
    print(f"Best validation loss: {best_val_loss:.6f}")

writer.close()

NameError: name 'run_name' is not defined

In [ ]:
!cp -r ./mlruns/* /content/drive/MyDrive/TCC/mlruns

In [ ]:
import os
import numpy as np
import rasterio
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import mlflow
import mlflow.pytorch
import matplotlib.pyplot as plt

def predict_geotiff(model, input_file, target_file, output_file, patch_size, stride, batch_size, device, run_name="EvaluationRun"):
    # --- Start MLflow run ---
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("patch_size", patch_size)
        mlflow.log_param("stride", stride)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("model_checkpoint", os.path.basename(output_file))

        inference_dataset = GeoTiffPatchDatasetShuffled(
            input_file, target_file,
            patch_size=patch_size, stride=stride, split='all'
        )
        inference_loader = DataLoader(inference_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        output_arr = np.zeros((inference_dataset.height, inference_dataset.width), dtype=np.float32)
        count_arr = np.zeros((inference_dataset.height, inference_dataset.width), dtype=np.int32)

        model.eval()
        mae_list, rmse_list = [], []

        with torch.no_grad():
            for inputs, targets, masks, coords in tqdm(inference_loader, desc="Predicting patches"):
                inputs, targets = inputs.to(device), targets.to(device)
                preds = model(inputs)

                outputs_np = preds.squeeze(1).cpu().numpy()
                targets_np = targets.squeeze(1).cpu().numpy()

                # Compute metrics for this batch
                mae = np.mean(np.abs(outputs_np - targets_np))
                rmse = np.sqrt(np.mean((outputs_np - targets_np) ** 2))
                mae_list.append(mae)
                rmse_list.append(rmse)

                # Merge patch predictions into the mosaic
                for i in range(outputs_np.shape[0]):
                    left, top = coords[0][i], coords[1][i]
                    output_arr[top : top + patch_size, left : left + patch_size] += outputs_np[i]
                    count_arr[top : top + patch_size, left : left + patch_size] += 1

        # Final averaged map
        output_arr /= np.maximum(count_arr, 1)

        # --- Compute global metrics ---
        mae_global = float(np.mean(mae_list))
        rmse_global = float(np.mean(rmse_list))
        mlflow.log_metric("MAE", mae_global)
        mlflow.log_metric("RMSE", rmse_global)

        print(f"✅ MAE: {mae_global:.4f}, RMSE: {rmse_global:.4f}")

        # --- Save predicted GeoTIFF ---
        output_meta = inference_dataset.input_meta.copy()
        output_meta.update({'count': 1, 'dtype': 'float32'})

        with rasterio.open(output_file, 'w', **output_meta) as dst:
            dst.write(output_arr, 1)

        print(f"Prediction saved to {output_file}")
        mlflow.log_artifact(output_file)

        # --- Visualization ---
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        sample_idx = 0
        rgb = inputs[sample_idx, :3].cpu().numpy().transpose(1, 2, 0)
        tgt = targets[sample_idx, 0].cpu().numpy()
        pred = preds[sample_idx, 0].cpu().numpy()

        axes[0].imshow(rgb)
        axes[0].set_title("Input RGB")
        axes[1].imshow(tgt, cmap='viridis')
        axes[1].set_title("Target")
        axes[2].imshow(pred, cmap='viridis')
        axes[2].set_title("Prediction")

        plt.tight_layout()
        fig_path = "evaluation_sample.png"
        plt.savefig(fig_path, dpi=300)
        mlflow.log_artifact(fig_path)
        plt.close(fig)

        print("🧾 Evaluation artifacts logged to MLflow.")

ModuleNotFoundError: No module named 'rasterio'

In [ ]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import segmentation_models_pytorch as smp

# Main prediction function
# def predict_geotiff(model, input_file, target_file, output_file, patch_size, stride, batch_size, device, name):
#     # Create dataset for inference (no train/val split needed)
#     inference_dataset = GeoTiffPatchDatasetShuffled(input_file, target_file, patch_size=patch_size, stride=stride, split='all')
#     inference_loader = DataLoader(inference_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

#     # Prepare output array
#     output_arr = np.zeros((inference_dataset.height, inference_dataset.width), dtype=np.float32)
#     count_arr = np.zeros((inference_dataset.height, inference_dataset.width), dtype=np.int32) # To handle overlaps

#     model.eval()
#     with torch.no_grad():
#         for inputs, targets, masks, coords in tqdm(inference_loader, desc="Predicting patches"):
#             inputs = inputs.to(device)
#             outputs = model(inputs)

#             # Move predictions to CPU and convert to numpy
#             outputs_np = outputs.squeeze(1).cpu().numpy() # Remove channel dimension

#             # Place predictions into the output array
#             for i in range(outputs_np.shape[0]):
#                 left, top = coords[0][i], coords[1][i] # Access coordinates correctly
#                 output_arr[top : top + patch_size, left : left + patch_size] += outputs_np[i]
#                 count_arr[top : top + patch_size, left : left + patch_size] += 1

#     # Average overlapping predictions
#     output_arr /= np.maximum(count_arr, 1) # Avoid division by zero

#     # Save the output GeoTIFF
#     output_meta = inference_dataset.input_meta.copy()
#     output_meta.update({
#         'count': 1,  # Single band output
#         'dtype': 'float32'
#     })

#     with rasterio.open(output_file, 'w', **output_meta) as dst:
#         dst.write(output_arr, 1)

#     print(f"Prediction saved to {output_file}")


# --- Main Execution ---
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load the trained model
    model = smp.DeepLabV3Plus(
      encoder_name="resnet101",        # backbone
      encoder_weights="imagenet",            # set to "imagenet" if you want pretrained weights
      in_channels=5,                   # your dataset has 5 input channels
      classes=1,                       # single regression/segmentation output
    )
    model = model.to(device)
    model.load_state_dict(torch.load('/content/drive/MyDrive/TCC/unetRandomSplit.pt'))
    print("Model loaded successfully.")

    input_file =  '/content/drive/MyDrive/TCC/data/input2021.tif'

    # Define output file path
    output_file = '/content/drive/MyDrive/TCC/data/amazonPrediction2021DeepLabV3.tif'

    # Run prediction
    predict_geotiff(model, input_file, target_file, output_file, patch_size, stride, batch_size, device,'Biomass_Prediction_Eval')

Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/179M [00:00<?, ?B/s]

Model loaded successfully.
Loading GeoTIFFs into memory (first time only)...
